In [2]:
import nltk
import re
import numpy as np
import pandas as pd
from pprint import pprint
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import spacy
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
import pickle
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
from nltk.stem import WordNetLemmatizer
import os, sys

C:\ProgramData\Anaconda3\lib\site-packages\past\types\oldstr.py:33: DeprecationWarning: invalid escape sequence \d
  """


In [4]:
data = pd.read_csv('abcnews-date-text.csv', error_bad_lines =False)
data['publish_date'] = pd.to_datetime(data['publish_date'].astype(str), format = '%Y%m%d')
data['publish_date'] = pd.DatetimeIndex(data['publish_date']).year

In [3]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
        
        
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

In [5]:
data_words = list(sent_to_words(data['headline_text']))
data_words_nostops = remove_stopwords(data_words)
data_words_nostops = pd.Series(data_words_nostops)
data_lemmatized = data_words_nostops.apply(lambda x: [WordNetLemmatizer().lemmatize(word, pos='v') for word in x])

In [19]:
with open('./pickle_data/data_lemmatized.pickle', 'wb') as f:
    pickle.dump(data_lemmatized, f, pickle.HIGHEST_PROTOCOL)

In [5]:
with open('data_lemmatized.pickle', 'rb') as f:
    data_lemmatized = pickle.load(f)

In [6]:
stop_words.extend(['interview', 'weather', 'abc', 'australia', 'australian','queensland', 'australias','sydney', 'nz', 'plan','get','back','new'])

In [7]:
stop_words.extend(['urge','group','council','say','report','two','qld','call','nsw','may','take','seek','police','make','tas'])

In [8]:
stop_words.extend(['meet','win','talk', 'govt','find','man','set','wa','continue','miss','three','one','two','welcome','fire','year','sa'])

In [ ]:
stop_words.extend(['melbourne','first','second','third','four','fourth','face','adelaide','government','govt','news','years','canberra','brisbane','offer',''])

In [9]:
data_lemmatized = remove_stopwords(data_lemmatized)

In [10]:
temp = data
temp['lemmatize'] = data_lemmatized
for i in range(15):
    globals()['trend{}'.format(i+2003)] = temp.loc[temp.publish_date == i+2003]

trend_list = [trend2003, trend2004, trend2005, trend2006, trend2007, trend2008, trend2009, trend2010, trend2011, trend2012, trend2013, trend2014, trend2015, trend2016, trend2017]

In [12]:
for i, year in enumerate(trend_list):    
    num_topics = 20
    passes = 10
    chunksize = 8000
    lemmatized = list(year['lemmatize'])
    id2word = corpora.Dictionary(lemmatized)
#     id2word.filter_extremes(no_below=5, no_above=0.3, keep_n=10000)
    texts = lemmatized
    corpus = [id2word.doc2bow(text) for text in texts]
    
    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=num_topics, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=chunksize,
                                           passes=passes,
                                           alpha='auto',
                                           per_word_topics=True)
    
    with open("./pickle_data_topic_20/result_"+str(i+2003)+".pickle", 'wb') as f:
        pickle.dump(lda_model.show_topics(num_topics=num_topics, num_words=20, log=False, formatted=True), f, pickle.HIGHEST_PROTOCOL)

    pyLDAvis.enable_notebook()
    prepared_data  = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
    pyLDAvis.save_html(prepared_data, './result_topic_20/LDA_result'+str(i+2003)+'.html')   